# Serverless Postgres Database
* Neon postgres as Database
* SQLAlchemy as ORM

In [1]:
%pip install psycopg2-binary
%pip install SQLAlchemy

  Obtaining dependency information for psycopg2-binary from https://files.pythonhosted.org/packages/7b/08/9c66c269b0d417a0af9fb969535f0371b8c538633535a7a6a5ca3f9231e2/psycopg2_binary-2.9.9-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 445.2 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 445.2 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 445.2 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 445.2 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 445.2 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 445.2 kB/s eta 0:00:03
   - -------------------------------------- 0.0/1.2 MB 85.6 kB/s eta 0:00:14
   -- ------------------------------------- 0.1/1.2 MB 170.3 kB/s eta 0:00:07
   -- ---------


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for SQLAlchemy from https://files.pythonhosted.org/packages/a3/6a/5a16b67f347584efbdf5bedcd5242155a9247c2c82972de2f59c80ae919f/SQLAlchemy-2.0.25-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for greenlet!=0.4.17 from https://files.pythonhosted.org/packages/53/80/3d94d5999b4179d91bcc93745d1b0815b073d61be79dd546b840d17adb18/greenlet-3.0.3-cp312-cp312-win_amd64.whl.metadata
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
    --------------------------------------- 0.0/2.1 MB 445.2 kB/s eta 0:00:05
    --------------------------------------- 0.0/2.1 MB 445.2 kB/s eta 0:00:05
    --------------------------------------- 0.0/2.1 MB 445.2 kB/s eta 0:00:05
    --------------------------------------- 0.0/2.1 MB 445.2 kB/s eta 0:00:05
    --------------------------------------- 0.0/


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from sqlalchemy import create_engine


In [21]:
import os
from dotenv import load_dotenv
from sqlalchemy.engine.base import Engine
load_dotenv()
NEON_DB_CONN_STR = os.getenv("NEON_DB_CONN_STR")


engine: Engine = create_engine(NEON_DB_CONN_STR)

# Getting Connection String

In [4]:
from sqlalchemy import text

with engine.connect() as conn:
    result = conn.execute(text("select 'hello world'"))
    print(result.all())

[('hello world',)]


# Create Table and Insert Data with Commit as You Go Transaction Style
* Create a table and insert some data, and the transaction is then committed using the Connection.commit() method, invoked inside the block where we acquired the Connection object:

# Create Table

In [26]:
from sqlalchemy import text

with engine.connect() as conn:
    # Start a new transaction
    trans = conn.begin()
    try:
        conn.execute(text("create TABLE IF NOT EXISTS students(id INTEGER, name VARCHAR)"))
        
        # Commit the transaction
        
        trans.commit()
    except Exception as e:
        # Rollback the transaction in case of error
        trans.rollback()
        raise e

# Data Insert

In [27]:
from sqlalchemy import text

with engine.connect() as conn:
    # Start a new transaction
    trans = conn.begin()
    try:
       
        conn.execute(
            text("INSERT INTO students (id, name) VALUES (:id, :name)"),
            [{"id": 1, "name": "Yasir"}, {"id": 2, "name": "Nasir" }]
        )
        # Commit the transaction
        
        trans.commit()
    except Exception as e:
        # Rollback the transaction in case of error
        trans.rollback()
        raise e

In [6]:
from sqlalchemy import text

with engine.connect() as conn:
    # Start a new transaction
    trans = conn.begin()
    try:
       
        conn.execute(
            text("INSERT INTO students (id, name) VALUES (:id, :name)"),
            [{"id": 3, "name": "Tahir"}, {"id": 4, "name": "Jafar" }]
        )
        # Commit the transaction
        
        trans.commit()
    except Exception as e:
        # Rollback the transaction in case of error
        trans.rollback()
        raise e

# Data Fetching

In [7]:
from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("SELECT id, name FROM students"))
    print(type(result))
    for student in result:
        print(f"id: {student.id}  name: {student.name}")

<class 'sqlalchemy.engine.cursor.CursorResult'>
id: 1  name: Yasir
id: 2  name: Nasir
id: 3  name: Tahir
id: 4  name: Jafar


In [8]:
from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("SELECT id, name FROM students"))
    print(type(result))
    for student in result:
        print(f"  name: {student.name}")

<class 'sqlalchemy.engine.cursor.CursorResult'>
  name: Yasir
  name: Nasir
  name: Tahir
  name: Jafar


In [33]:
from sqlalchemy import text
with engine.connect() as conn:
        result  = conn.execute(text("SELECT name FROM students WHERE id = 4"), 
                               {"id":3} )
        for student in result:
            # print(f"id: {student.id}, name: {student.name}")
            print(f"Name: {student.name}")

Name: Jafar


# Add Column to the Existing Table
* using Alter Table 

In [22]:
from sqlalchemy import text

with engine.connect() as conn:
    # Start a new transaction
    trans = conn.begin()
    try:
        # Add a new column 'age' to the 'students' table
        conn.execute(text("ALTER TABLE students ADD COLUMN age INTEGER"))
        
        # Commit the transaction
        trans.commit()
    except Exception as e:
        # Rollback the transaction in case of error
        trans.rollback()
        raise e


# Update Values to newly Created Column in Student Table
* Using update

In [31]:
from sqlalchemy import text

with engine.connect() as conn:
    # Start a new transaction
    trans = conn.begin()
    try:
        conn.execute(text("UPDATE students SET age = :new_age WHERE id = :student_id"),
                     {"new_age": 15, "student_id": 3})
                    
        
        # Commit the transaction
        trans.commit()
    except Exception as e:
        # Rollback the transaction in case of error
        trans.rollback()
        raise e


In [44]:
from sqlalchemy import text
with engine.connect() as conn:
        result  = conn.execute(text("SELECT * FROM students WHERE id >= :id1 AND id <= :id2"), 
                               {"id1":2, "id2":3} )
        for student in result:
            print(f"ID: {student.id}, Name: {student.name}, Age: {student.age}")
            

ID: 2, Name: Nasir, Age: 30
ID: 3, Name: Tahir, Age: 15
